# 第3章 TensorFlow入门

## 3.4 TensorFlow实现神经网络

### 3.4.3 神经网络参数与TensorFlow变量

In [1]:
# 通过变量实现神经网络的参数并实现前向传播的过程
import tensorflow as tf

w1 = tf.Variable(tf.random_normal((2, 3), stddev=1, seed=1), name='w1')
w2 = tf.Variable(tf.random_normal((3, 1), stddev=1, seed=1), name='w2')
x = tf.constant([[0.7, 0.9]])

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()
# 对w1和w2显式运行初始化过程
sess.run(w1.initializer)
sess.run(w2.initializer)
print sess.run(y)
sess.close()

Instructions for updating:
Colocations handled automatically by placer.
[[3.957578]]


In [2]:
# 查看当前所有变量
tf.global_variables()

[<tf.Variable 'w1:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'w2:0' shape=(3, 1) dtype=float32_ref>]

- 类似张量，维度（shape）和类型（type）也是变量最重要的两个属性，一个变量在构建之后，类型不可变，但是维度可以改变（通常不这样做），需设置参数`validate_shape=False`

In [3]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1), name='w1')
w2 = tf.Variable(tf.random_normal([2, 2], stddev=1), name='w2')
# 以下语句报错
tf.assign(w1, w2, validate_shape=False)

<tf.Tensor 'Assign:0' shape=(2, 2) dtype=float32_ref>

### 3.4.4 通过TensorFlow训练神经网络模型

In [4]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal([2, 3], seed=1, stddev=1))
w2 = tf.Variable(tf.random_normal([3, 1], seed=1, stddev=1))

# 定义placeholder作为存放输入数据的地方，这里的维度不一定要定义，但如果维度是确定的，给出维度可以降低出错概率
#x = tf.placeholder(tf.float32, shape=(1, 2), name='input')
x = tf.placeholder(tf.float32, shape=(3, 2), name='input') # 每次提供一个batch的训练样例
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# feed_dict是一个字典，给出每个用到的placeholder的取值，如果某个需要的placeholder没有被指定取值，会报错
#print sess.run(y, feed_dict={x: [[0.7, 0.9]]})
print sess.run(y, feed_dict={x: [[0.7, 0.9], [0.1, 0.4], [0.5, 0.8]]})

[[3.957578 ]
 [1.1537654]
 [3.1674924]]


### 3.4.5 完整神经网络样例程序

In [5]:
import tensorflow as tf
from numpy.random import RandomState

# 定义训练数据的batch大小和学习率
batch_size = 8
learning_rate = 0.001

# 定义神经网络的参数，这里沿用之前使用的网络结构
w1 = tf.Variable(tf.random_normal([2, 3], seed=1, stddev=1))
w2 = tf.Variable(tf.random_normal([3, 1], seed=1, stddev=1))

# 在shape的一个维度上使用None可以方便使用不同的batch大小：训练时采用小的batch，预测时如果数据集较小可以使用全部数据
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

# 前向传播
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 定义损失函数和反向传播算法
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)) + (1-y) * tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy) # Adam优化算法

# 随机生成正负样本
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

# 创建一个会话来运行TensorFlow程序
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    # 初始化变量
    sess.run(init_op)
    # 打印出训练之前神经网络的参数值（随机初始化所得）
    print sess.run(w1)
    print sess.run(w2)
    # 设定训练的轮数
    STEPS = 5000
    for i in range(STEPS):
        # 每次选取batch_size个样本进行训练
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        # 通过选取的样本训练神经网络并更新参数
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        # 每隔一段时间计算在所有数据上的交叉熵并输出
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print "After %d training steps, cross entropy on all data is %g" % (i, total_cross_entropy)
            
    # 训练之后神经网络的参数值
    print sess.run(w1)
    print sess.run(w2)

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training steps, cross entropy on all data is 0.314006
After 1000 training steps, cross entropy on all data is 0.0684551
After 2000 training steps, cross entropy on all data is 0.0337151
After 3000 training steps, cross entropy on all data is 0.020558
After 4000 training steps, cross entropy on all data is 0.0136866
[[-2.5486584  3.0793116  2.8951745]
 [-4.111278   1.6259075  3.3972738]]
[[-2.3230944]
 [ 3.30117  ]
 [ 2.463214 ]]


- 以上程序实现了训练神经网络的全部过程，从这段程序可以总结出训练神经网络的过程可以分为以下三个步骤：1.定义神经网络的结构和前向传播的输出结果；2.定义损失函数以及选择反向传播优化的算法；3.生成会话（tf.Session）并且在训练数据上反复运行反向传播优化算法
- 无论神经网络的结构如何变化，这三个步骤是不变的

## 小结

- 本章介绍了TensorFlow的三个基本概念：计算图（tf.Graph）、张量（tf.Tensor）和会话（tf.Session）
- 计算图是TensorFlow的计算模型，所有TensorFlow的程序都会通过计算图的形式表示，计算图上的每一个节点都是一个运算，而计算图上的边则表示了运算之间的数据传递关系，计算图上还保存了运行每个运算的设备信息（CPU or GPU）以及运算之间的依赖关系；计算图提供了管理不同集合的功能，并且TensorFlow会自动维护5个不同的默认集合
- 张量是TensorFlow的数据模型，TensorFlow中所有运算的输入、输出都是张量，张量本身并不存储任何数据，它只是对运算结果的引用，通过张量，可以更好地组织TensorFlow程序
- 会话是TensorFlow的运算模型，它管理了一个TensorFlow程序拥有的系统资源，所有的运算都要通过会话执行